In [1]:
import os
import pandas as pd
from torch.utils.data import TensorDataset, random_split
import matplotlib.pyplot as plt
import torch.nn.functional as nnf

In /home/fatma/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/fatma/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/fatma/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/fatma/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/fatma/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylel

# Acculmating the attention scores of BERT with Fine tuning

In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2080 with Max-Q Design


In [3]:
from transformers import *
from torch.utils.data import TensorDataset, random_split, SequentialSampler,DataLoader, RandomSampler
import pandas as pd
import numpy as np
from collections import defaultdict
# (logits, hidden_states, attentions)
import seaborn as sns; sns.set()

/home/fatma/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/fatma/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/fatma/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/fatma/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

## Get attention function for a certain sentence

In [4]:


def get_model_attention_for_sentence(sentences,single_sentence=False, add_special_token=False):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  attention_weights_per_sentence = []
  sentences_tokens = []
  if single_sentence == True:
    token = [tokenizer.cls_token] + tokenizer.tokenize(sentences) + [tokenizer.sep_token]
    inputids = torch.LongTensor([tokenizer.encode(token, add_special_tokens=add_special_token)])   
    # Add the encoded sentence to the list.    
    outputs = model(inputids)   # Forward pass, calculate logit predictions
    sentences_tokens.append(token)
    attention_weights_per_sentence.append(outputs[2])
  else:
    # For every sentence...
    for sent in sentences:
        token = [tokenizer.cls_token] + tokenizer.tokenize(sent) + [tokenizer.sep_token]
        inputids = torch.LongTensor([tokenizer.encode(token, add_special_tokens=add_special_token)])   
        # Add the encoded sentence to the list.    
        outputs = model(inputids)   # Forward pass, calculate logit predictions
        sentences_tokens.append(token)
        attention_weights_per_sentence.append(outputs[2])
  return sentences_tokens, attention_weights_per_sentence

In [5]:
def get_attention_map_per_word_as_df (sentences_tokenized, attention_list_per_sentence):
  #get attention map for words in sentences
  attention_map_per_word = defaultdict(list)
  for token_sentence in sentences_tokenized:
    sentence_index = sentences_tokenized.index(token_sentence)
    #print(token_sentence)  
    for token in token_sentence:
      word_index = token_sentence.index(token)
      #print(token)
      for layer in range(0,12):
        for head in range(0,12):
          attention_weight_per_layer_per_head_per_word = attention_list_per_sentence[sentence_index][layer][0][head]
          normalized_attention_weight_per_layer_per_head_per_word = torch.sum(attention_weight_per_layer_per_head_per_word,dim=0)/attention_weight_per_layer_per_head_per_word.shape[0]
          #print(len(normalized_attention_weight_per_layer_per_head_per_word))
          attention_map_per_word["word"].append(token)
          attention_map_per_word["attention_map"].append({layer+1:{head+1:float(normalized_attention_weight_per_layer_per_head_per_word[word_index])}})
  attention_map_per_word_df = pd.DataFrame.from_dict(attention_map_per_word, orient='index')
  attention_map_per_word_df = attention_map_per_word_df.transpose()       
  return attention_map_per_word_df           

In [6]:
def generate_attention_attention_layers_per_head_for_word(word,attention_map_df):
  import ast
  frames = []
  #len(attention_map_df[attention_map_df["word"]==word]["attention_map"].values)
  for i in attention_map_df[attention_map_df["word"]==word]["attention_map"].values:
    frames.append(pd.DataFrame.from_dict(i, orient='index'))
  res = pd.concat(frames)
  res = res.fillna(0)
  res = res.groupby(res.index).sum()
  return res

In [7]:
def generate_attention_attention_layers_per_head_for_word(word,attention_map_df):
  frames = []
  #len(attention_map_df[attention_map_df["word"]==word]["attention_map"].values)
  for i in attention_map_df[attention_map_df["word"]==word]["attention_map"].values:
    frames.append(pd.DataFrame.from_dict(i, orient='index'))
  res = pd.concat(frames)
  res = res.fillna(0)
  res = res.groupby(res.index).sum()
  return res

In [8]:
def generate_attention_heat_map_for_word(word,attention_map_df):
  frames = []
  for i in attention_map_df[attention_map_df["word"]==word]["attention_map"].values:
    frames.append(pd.DataFrame.from_dict(i, orient='index'))
  res = pd.concat(frames)
  res = res.fillna(0)
  res = res.groupby(res.index).sum()
  ax = sns.heatmap(res, cbar=True)
  ax.set(xlabel='head', ylabel='layer')
  ax.set_title("word '"+str(word)+"'")
  ax.invert_yaxis()

In [9]:
def generate_attention_layer_avg_line_plot_for_word(word,attention_map_df):
  import seaborn as sns; sns.set()
  frames = []
  for i in attention_map_df[attention_map_df["word"]==word]["attention_map"].values:
    frames.append(pd.DataFrame.from_dict(i, orient='index'))
  res = pd.concat(frames)
  df = res.fillna(0)
  res = res.groupby(res.index).sum()
  avg_head_res = res.sum(axis=1).values
  layers = res.index.values
  plt.title("word '"+str(word)+"'")
  plt.xticks(np.arange(12))
  plt.xlabel("layers")
  plt.ylabel("avg. attention weight")
  #print(avg_head_res)
  plt.plot(layers,avg_head_res)
  plt.grid(False)
  plt.show()
  #ax = sns.lmplot("x","y",data=res)
  #ax.set(xlabel='head', ylabel='layer')
  #ax=sns.catplot(kind="swarm",x=res.index,y=res.values)
  #ax.invert_yaxis()

## Read Data

In [10]:
import torch
#output_dir = 'bert-base-uncased'
output_dir = "../../trained_models/BERT-Fine-Tuned/Pytorch/Fine_Tune_Kaggle_clean_text128/"

config = BertConfig.from_pretrained(output_dir, output_hidden_states=True, output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(output_dir, do_lower_case=True)

model = BertForSequenceClassification.from_pretrained(output_dir, config=config)

In [11]:
# Load the dataset into a pandas dataframe.
data_test = pd.read_csv("../../Data/Kaggle/kaggle_data_test.csv)
data_test = data_test.dropna()

In [ ]:
sentences_lst_tn = data_test.sample(1000).Text_clean.values
sentences_tokenized_lst_tn, attention_list_per_sentence_lst_tn = get_model_attention_for_sentence(sentences_lst_tn)
attention_map_per_word_lst_tn_df = get_attention_map_per_word_as_df(sentences_tokenized_lst_tn, attention_list_per_sentence_lst_tn )

In [ ]:
attention_map_per_word_lst_tn_df.to_csv("attention_maps/bert_with_fine_tuning/Kaggle_BERT_Fine_Tuning_1000_data_attetnion_weights"")